<a href="https://colab.research.google.com/github/billzhao1030/KIT315_Project/blob/KIT/KIT315_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Introduction***

In this KIT315 project, I developed an machine learning application to help us detecting and classifying the type of Penguin from images to help Antarctic scientific 
research. There are 
three categories I need to classify: ***Aptenodytes Forsteri (Emperor Penguin)***, ***Aptenodytes Patagonicus (King Penguin)*** and  ***Pygoscelis 
Antarciticus (Chinstrap Penguin)*** .
<Br>
<Br>
In this jupyter notebook, I will describe the most important tasks I did during the ML application development, these include the data preparation, processing, and analysis, model development, evaluation and selection, and how I apply the model.
<Br>
<p align="justify"> In this jupyter notebook, I will describe the most important tasks I did during the ML application development, these include the data preparation, processing, and analysis, model development, evaluation and selection, and how I apply the model. </p>
<Br>
Submission details:
*   Student Name: Xunyi Zhao (Bill)
*   Student ID: 560060
*   Date: 04/10/2022
*   Environment uses: [Google Colab](https://colab.research.google.com/)

# ***Data Preparation***



In this section I will introduce how I collect and 
annotate the data (using the [Roboflow](https://roboflow.com/) tool) which helps achieve good 
performance of the model development.


# ***Data Processing***

In this section, I will introduce how I identify if data processing is needed or not, and apply relevant techniques for data pre-processing using the [Roboflow](https://roboflow.com/) tool.


# ***Data Analysis***

In this section, I will perform some analysis of the data, this includes:

*   Number of samples.
*   Number of attributes/size.
*   Is the data balance or not.
*   Are there any missing values?
*   The chanllenges for learning with this data.


# ***Model Development***

In this section, there are **three** we need to develop:


1.   Yolov5
2.   Detectron2-Faster RCNN
3.   TODO


Now we need to install the ***Roboflow*** to load the data (in different format)

In [ ]:
!pip install roboflow
from roboflow import Roboflow

## ***YOLOv5 Model***

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone yolov5
%cd yolov5
%pip install -r requirements.txt  # install

import utils
display = utils.notebook_init() # check

YOLOv5 🚀 v6.2-89-g5cb9fe6 Python-3.7.13 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 37.4/78.2 GB disk)


In [ ]:
rf = Roboflow(api_key="pCjhgO6AaDWtsuKCrLgf")
project = rf.workspace("utas").project("kit315-dequm")
dataset = project.version(5).download("yolov5")

In [ ]:
!python train.py --img 640 --batch 16 --epochs 60 --data KIT315-5/data.yaml --weights yolov5s.pt --cache

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/train

In [ ]:
!python detect.py --weights ./runs/train/exp/weights/best.pt  --conf 0.3 --source ./KIT315-5/test/images/

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cp ./runs/train/exp/weights/best.pt /content/gdrive/My\ Drive/models/yolov5_tutorial_model

## ***Detectron2-Faster RCNN Model***

In [ ]:
!python -m pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# !pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
# !pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
# import torch, torchvision
# print(torch.__version__, torch.cuda.is_available())
# !gcc --version

# !pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="pCjhgO6AaDWtsuKCrLgf")
project = rf.workspace("utas").project("kit315-dequm")
dataset = project.version(5).download("coco")

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/KIT315-5/train/_annotations.coco.json", "/content/KIT315-5/train")
register_coco_instances("my_dataset_val", {}, "/content/KIT315-5/valid/_annotations.coco.json", "/content/KIT315-5/valid")
register_coco_instances("my_dataset_test", {}, "/content/KIT315-5/test/_annotations.coco.json", "/content/KIT315-5/test")

In [ ]:
#visualize training data
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

import random
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
#from .detectron2.tools.train_net import Trainer
#from detectron2.engine import DefaultTrainer
# select from modelzoo here: https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md#coco-object-detection-baselines

from detectron2.config import get_cfg
#from detectron2.evaluation.coco_evaluation import COCOEvaluator
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
# faster_rcnn_R_101_FPN_3x.yaml
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

cfg.DATALOADER.NUM_WORKERS = 2 # Max 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4 # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.001


cfg.SOLVER.MAX_ITER = 1200 #adjust up if val mAP is still rising, adjust down if overfit
# cfg.SOLVER.STEPS = (1000, 1500)
# cfg.SOLVER.GAMMA = 0.05
cfg.SOLVER.STEPS = [] # do not decay learning rate


cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4 #your number of classes + 1

cfg.TEST.EVAL_PERIOD = 500

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")

In [ ]:
from detectron2.utils.visualizer import ColorMode
import glob

for imageName in glob.glob('/content/KIT315-5/test/*jpg'):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata, 
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])

## ***Third Model***

# ***Model Evaluation and Selection***

## ***Train the best model***

Based on the model evaluation and selection steps mentioned in previous sections, now I will train the best model - TODO, and save it for later use.

# ***Apply Model***

In this last section, I will apply the best model I've developed in previous steps to the new data (the samples mainly represent two different scenarios which are ***images*** and ***videos***).